In [ ]:
import numpy as np
import plotly.graph_objects as go
import matplotlib.pyplot as plt

In [ ]:
%reload_ext yamlmagic
%matplotlib inline

In [ ]:
import warnings
import importlib
import simtool
from IPython.display import display

class WidgetConstructor():
    def __init__(self, layx=None):
        self.inputs = {}
        self.outputs = {}
        self.layout = None
        for item, value in layx.items():
            if item == "inputs":
                self.buildInputs(value)  
            elif item == "outputs":
                self.buildOutputs(value)  
            elif item == "layout":
                self.layout = self.buildLayout(value)  
            else:
               pass;#warnings.warn(item + " is ignored")  
    
    def buildInputs(self, inputs):
        for item, value in inputs.items():
            if type(value) == dict:
                type_ = ""
                module_ = "ipywidgets"
                params_ = {}
                click_ = None
                for item2, value2 in value.items():
                    if item2 == "type":
                        type_ = value2;
                    elif item2 == "click":
                        click_ = value2;
                    elif item2 == "module":
                        module_ = value2;
                    else:
                        params_[item2] = value2
                module = importlib.import_module(module_)
                class_ = getattr(module, type_)
                self.inputs[item] = class_(**params_)  
                self.setClick(self.inputs[item], click_)
            else:
               warnings.warn(item + " is not a valid description")  
            
    def buildOutputs(self, outputs):
        for item, value in outputs.items():
            if type(value) == dict:
                type_ = ""
                module_ = "ipywidgets"
                params_ = {}
                click_ = None
                for item2, value2 in value.items():
                    if item2 == "type":
                        type_ = value2;
                    elif item2 == "click":
                        click_ = value2;
                    elif item2 == "module":
                        module_ = value2;
                    else:
                        params_[item2] = value2
                module = importlib.import_module(module_)
                class_ = getattr(module, type_)
                self.outputs[item] = class_(**params_)   
                self.setClick(self.outputs[item], click_)                
            else:
               warnings.warn(item + " is not a valid description")  


    def buildLayout(self, layout):
        type_ = ""
        module_ = "ipywidgets"
        params_ = {}
        children_ = []
        titles_ = []
        click_ = None
        for item, value in layout.items():
            if item == "type":
                type_ = value;
            elif item == "module":
                module_ = value;
            elif item == "click":
                click_ = value;                
            elif item == "titles":
                titles_ = value
            elif item == "children":
                if type(value) == dict:
                    for item2, value2 in value.items():
                        child_ = None
                        if value2 == None:
                            if (item2 in self.inputs):
                                child_ = self.inputs[item2]
                            elif (item2 in self.outputs):
                                child_ = self.outputs[item2]
                            else :
                               warnings.warn(item2 + " is not a valid element")
                        else:
                            child_ = self.buildLayout(value2)
                        if child_ is not None:
                            children_.append(child_)
                else:
                   warnings.warn(item + " is not a valid description")

            else:
                params_[item] = value
        if (len(titles_) > 0 ):
            params_["titles"] = titles_
        if (len(children_) > 0 ):
            params_["children"] = children_
        module = importlib.import_module(module_)
        class_ = getattr(module, type_)
        instance_ =  class_(**params_) 
        self.setClick(instance_, click_)
        
        for i, title in enumerate(titles_):
            try:
                instance_.set_title(i, title)
            except:
               warnings.warn(title + " can not be assigned")

        return instance_
         
    def setClick(self, instance, function_name):
        if function_name is not None:
            instance.on_click(None, remove=True)
            instance.on_click(lambda a, b=self, c=function_name: getattr(b, c)(b))
               
        
    def display(self):
        display(self.layout)

def simtool_constructor(self, node):
    values = node.value.split(" ", 2)
    tool = values[0]
    path = ""
    action = "values"
    if len(values) > 1:
        path = values[1]
    if len(values) > 2:
        action = values[2]        
    stl = simtool.searchForSimTool(tool)
    if (stl['notebookPath'] == None):
        raise "Simtool is not valid"
    inputs = simtool.getSimToolInputs(stl)
    outputs = simtool.getSimToolOutputs(stl)
    res = {'inputs':{},'outputs':{}}
    for i in inputs:
        res['inputs'][i] = {}
        for j in inputs[i]:
            if inputs[i][j] is None:
                pass;
            elif j == "type":
                if inputs[i][j] == "Integer":
                    res['inputs'][i][j] = "BoundedIntText"
                elif inputs[i][j] == "Number":
                    res['inputs'][i][j] = "BoundedFloatText"
                elif inputs[i][j] == "Boolean":
                    res['inputs'][i][j] = "ToggleButton"
                else:
                    res['inputs'][i][j] = "Text"
            else :
                res['inputs'][i][j] = inputs[i][j]
            
    for i in outputs:
        res['outputs'][i] = {}
        for j in outputs[i]:
            if j == "type":
                res['outputs'][i][j] = "Output"
            else:
                res['outputs'][i][j] = outputs[i][j]
    for subpath in path.split("."):
        res = res.get(subpath, {})
    if action == "keys":
        return {k:None for k in res.keys()}
    else :
        return res
              

In [ ]:
%%yaml
defaults: &defaults
   layout:  
       width : auto

inputs:
    button1:
        type : Button  
        click : DEMO
        description : DEMO
        
    tb : 
        type: Togglebuttons
        name: Is this a DEMO
        description: Is this a DEMO
        value: 2
        options: 
            YES: 1
            Maybe: 2
            NO: 2
        module: hublib.ui
outputs:
    out1 : 
        type: FigureWidget
        module: plotly.graph_objects
    map1:
        type : Map
        module: ipyleaflet
        
    molecule1:
        type: NGLWidget
        module: nglview
        
    molecule2:
        type: Speck
        module: ipyspeck.speck

    plot1:
        type: plot
        module: k3d
    sheet: 
        type: sheet
        module: ipysheet
    output1:
        type: Output
layout:
    type: VBox
    border: 1px solid
    children:
        button1:
        tb:
        container: 
            type: Tab #Accordion
            children:
                molecules:
                    #type: VBox
                    type: GridBox
                    layout : 
                        grid_template_columns : 33% 33% 33%
                    children:
                        molecule1:
                        plot1:
                        molecule2:
                out1:            
                map1:
                sheet:
                output1:
            titles: [
                nglview - k3d - ipyspeck, 
                plotly, 
                ipyleaflet,
                sheet,
                Output
            ]


In [ ]:
bas = _

In [ ]:
import nglview
import pythreejs
import ipyleaflet
import k3d
import k3d.platonic as platonic
import numpy as np
import pyvista as pv
from pyvista import examples



def DEMO(widget):
    molecule1 = widget.outputs["molecule1"]
    molecule1.layout.width = "100%"
    molecule1.clear()
    molecule1.add_component('1hel.pdb')
    molecule1.add_surface(
        boxSize=10,
        wrap=True,
        color="skyblue",
        contour=True
    )
    
    molecule2 = widget.outputs["molecule2"]
    molecule2.data = '3\nWater molecule\nO 0.00000 0.00000 0.11779\nH 0.00000 0.75545 -0.47116\nH 0.00000 -0.75545 -0.47116'
    molecule2.frontview()
    
    map1 = widget.outputs["map1"]
    map1.center=(40.4237, -86.9212)
    map1.zoom=15
    marker = ipyleaflet.Marker(location=(40.4237, -86.9212), draggable=False, title="Purdue University")
    map1.add_layer(marker);


    x = np.random.randn(100,2).astype(np.float32)
    point_size = 0.2
    plot = widget.outputs["plot1"]
    plot.reset_camera()
    mesh = platonic.Dodecahedron().mesh
    plot.__iadd__(mesh) 

    out1 = widget.outputs["out1"]
    out1.data=[]
    out1.add_trace(
        go.Scatter(x=x[:,0], y=x[:,1], mode='markers')
    )
    
    output1 = widget.outputs["output1"]
    output1.clear_output()

    with output1:
        mesh = examples.download_bunny()
        mesh.flip_normals()
        pl = pv.Plotter()
        #pl = pv.PlotterITK()
        pl.add_mesh(mesh, color='lightgrey')
        pl.camera_position = 'xy'
        #pl.show(jupyter_backend='pythreejs')
        #pl.show()
        print("Hello world")


In [ ]:
s = WidgetConstructor(bas)
s.DEMO = DEMO
s.display()